In [3]:
import requests
import csv
import json
import socket 
from datetime import datetime

def get_block_info(block_hash):
    try:
        response = requests.get(f"https://blockchain.info/rawblock/{block_hash}")
        response.raise_for_status()
        return response.json()
    except requests.HTTPError as e:
        print(f"Ошибка HTTP: {e}")
        return None
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

def check_blacklist(btc_address):
    try:
        hostname = f"{btc_address}.bl.btcblack.it"
        ip_address = socket.gethostbyname(hostname)
        return ip_address == "127.0.0.2"
    except socket.gaierror:
        return False

def is_suspicious(addreses):
    for adress in addreses:
        if check_blacklist(adress):
            return True, adress
    return False, None

def process_transactions(transactions, csv_writer):
    for tx in transactions:
        inputs = {inp["prev_out"]["addr"]: inp["prev_out"]["value"] / 100000000 for inp in tx["inputs"] if "prev_out" in inp and "addr" in inp["prev_out"]}
        outputs = {out["addr"]: out["value"] / 100000000 for out in tx["out"] if "addr" in out}
        total_amount = sum(outputs.values())
        suspicious, address = is_suspicious(list(inputs.keys()) + list(outputs.keys()))
        if suspicious:
            print(f"\nОбнаружена транзакция: {tx['hash']}")
            print(f"Подозрительный адрес: {address}")
            print(f"Количество входов: {len(inputs)}, Количество выходов: {len(outputs)}, Total Amount: {total_amount} BTC")
        tx_details = {
            "Timestamp": (tx["time"]),
            "TotalAmount": total_amount,
            "TotalInputs": len(inputs),
            "TotalOutputs": len(outputs),
            "Inputs": json.dumps(inputs),
            "Outputs": json.dumps(outputs),
            "Suspicius": suspicious,
        }
        csv_writer.writerow(tx_details)

def fetch_transactions_and_write_csv(last_n_blocks=80, csv_filename="transactions.csv"):
    try:
        latest_block_hash = requests.get("https://blockchain.info/latestblock").json()["hash"]

        with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
            fieldnames = ["Timestamp", "TotalAmount", "TotalInputs", "TotalOutputs", "Inputs", "Outputs", "Suspicius"]
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            
            for _ in range(last_n_blocks):
                block_data = get_block_info(latest_block_hash)
                if block_data:
                    print(f"\n\n=== Анализ блока #{block_data['height']}, количество транзакций: {len(block_data['tx'])} ===\n\n")
                    process_transactions(block_data["tx"], writer)
                    latest_block_hash = block_data["prev_block"]
                else:
                    print("Ошибка получения данных о блоке.")
                    break
    except Exception as e:
        print(f"Ошибка: {e}")

fetch_transactions_and_write_csv()



=== Анализ блока #836973, количество транзакций: 3533 ===



Обнаружена транзакция: b6b8737e8c997ede8278993e1eeb787129dfc63cadf04cab880c661f4073fecf
Подозрительный адрес: 1Kr6QSydW9bFQG1mXiPNNu6WpJGmUa9i1g
Количество входов: 1, Количество выходов: 2, Total Amount: 7.970021119999999 BTC

Обнаружена транзакция: 6a189f0a302207f3846b21b58b32942edbe704a0093f287996d86ba28f6e0f4e
Подозрительный адрес: 35iMHbUZeTssxBodiHwEEkb32jpBfVueEL
Количество входов: 1, Количество выходов: 2, Total Amount: 0.011993130000000001 BTC

Обнаружена транзакция: 95ebfc4a4af83125699be8ed83d79abc95471aea9679b17886d20851d8fdbc1c
Подозрительный адрес: 37jAAWEdJ9D9mXybRobcveioxSkt7Lkwog
Количество входов: 4, Количество выходов: 1, Total Amount: 0.00448515 BTC

Обнаружена транзакция: 83a9f3e74b883d9dd0498cc96d9562fcde918b975a4acceeec476bacde347c8a
Подозрительный адрес: 37jAAWEdJ9D9mXybRobcveioxSkt7Lkwog
Количество входов: 10, Количество выходов: 1, Total Amount: 0.52984292 BTC

Обнаружена транзакция: 6b87ef035d82b7cf

KeyboardInterrupt: 

Код собирает данные о последних 80 блоков блокчейна биткойна. Записывает основные параметры транзакций в csv файл.

Информацию о кошельков мы берем с сайта https://btcblack.it/

In [5]:
import pandas as pd
import json
import numpy as np
import socket 
from datetime import datetime

df = pd.read_csv('transactions.csv')

wallets = {}

def check_blacklist(btc_address):
    try:
        hostname = f"{btc_address}.bl.btcblack.it"
        ip_address = socket.gethostbyname(hostname)
        return ip_address == "127.0.0.2"
    except socket.gaierror:
        return False

# Функция для обновления агрегированных данных кошелька
def update_wallet(wallet, amount, is_input, inputs, outputs, row):
    timestamp = datetime.fromtimestamp(row['Timestamp'])
    if wallet not in wallets:
        wallets[wallet] = {'total_sent': 0, 'total_received': 0, 'tx_amounts': [],
                           'tx_timestamps': [], 'unique_addresses': set(),
                           'total_inputs': 0, 'total_outputs': 0, 'tx_counts': 0,
                           'blacklisted': check_blacklist(wallet)}
    
    if is_input:
        wallets[wallet]['total_received'] += amount
    else:
        wallets[wallet]['total_sent'] += amount

    wallets[wallet]['tx_amounts'].append(amount)
    wallets[wallet]['tx_timestamps'].append(timestamp)
    wallets[wallet]['unique_addresses'].update(inputs.keys())
    wallets[wallet]['unique_addresses'].update(outputs.keys())
    wallets[wallet]['total_inputs'] += row['TotalInputs']
    wallets[wallet]['total_outputs'] += row['TotalOutputs']
    wallets[wallet]['tx_counts'] += 1

# Обработка каждой строки в датасете
for index, row in df.iterrows():
    if index % 100 == 0:  # Вывод информации каждые 100 строк
        print(f"Обрабатывается строка {index} из {len(df)}")


    inputs = json.loads(row['Inputs'])
    outputs = json.loads(row['Outputs'])
    total_amount = row['TotalAmount']
    
    for wallet in inputs:
        update_wallet(wallet, total_amount, True, inputs, outputs, row)

    for wallet in outputs:
        update_wallet(wallet, total_amount, False, inputs, outputs, row)

# Расчет интервалов между транзакциями
def calculate_intervals(timestamps):
    timestamps.sort()
    intervals = [(timestamps[i] - timestamps[i-1]).total_seconds() for i in range(1, len(timestamps))]
    return intervals if intervals else [0]

# Преобразование словаря в DataFrame и расчет дополнительных признаков
wallets_df = pd.DataFrame.from_dict(wallets, orient='index')
wallets_df['wallet'] = wallets_df.index
wallets_df['unique_address_count'] = wallets_df['unique_addresses'].apply(len)
wallets_df['avg_interval_between_tx'] = wallets_df['tx_timestamps'].apply(calculate_intervals).apply(np.mean)
wallets_df['avg_tx_amount'] = wallets_df['tx_amounts'].apply(np.mean)
wallets_df['max_tx_amount'] = wallets_df['tx_amounts'].apply(max)
wallets_df['min_tx_amount'] = wallets_df['tx_amounts'].apply(min)
wallets_df['avg_inputs'] = wallets_df['total_inputs'] / wallets_df['tx_counts']
wallets_df['avg_outputs'] = wallets_df['total_outputs'] / wallets_df['tx_counts']

# Очистка от временных колонок
wallets_df.drop(columns=['tx_amounts', 'tx_timestamps', 'unique_addresses', 'total_inputs', 'total_outputs', 'tx_counts'], inplace=True)

# Сохранение обработанного датасета в файл CSV
wallets_df.to_csv('wallets.csv', index=False)

print("Датасет кошельков сохранен в файл 'wallets.csv'.")


Обрабатывается строка 0 из 13800
Обрабатывается строка 100 из 13800
Обрабатывается строка 200 из 13800
Обрабатывается строка 300 из 13800
Обрабатывается строка 400 из 13800
Обрабатывается строка 500 из 13800
Обрабатывается строка 600 из 13800
Обрабатывается строка 700 из 13800
Обрабатывается строка 800 из 13800
Обрабатывается строка 900 из 13800
Обрабатывается строка 1000 из 13800
Обрабатывается строка 1100 из 13800
Обрабатывается строка 1200 из 13800
Обрабатывается строка 1300 из 13800
Обрабатывается строка 1400 из 13800
Обрабатывается строка 1500 из 13800
Обрабатывается строка 1600 из 13800
Обрабатывается строка 1700 из 13800
Обрабатывается строка 1800 из 13800
Обрабатывается строка 1900 из 13800
Обрабатывается строка 2000 из 13800
Обрабатывается строка 2100 из 13800
Обрабатывается строка 2200 из 13800
Обрабатывается строка 2300 из 13800
Обрабатывается строка 2400 из 13800
Обрабатывается строка 2500 из 13800
Обрабатывается строка 2600 из 13800
Обрабатывается строка 2700 из 13800
Обра

Код анализирует собранные данные о транзакциях, делая из них датасет кошельков. 
Нами были выбраны следующие параметры кошельков:
1) Уникальное количетсво адресов, с которыми проводилась операция
2) Средний интервал между транзакциями
3) Средний размер транзакции
4) Максимальный размер транзакции
5) Минимальный размер транзакции
6) Среднее количество inputs в транзакциях с этим кошельком
7) Среднее количество outputs в транзакциях с этим кошельком
8) Есть ли кошелек в блеклисте. 